In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

import analyze

In [ ]:
# sim_name = "single_antenna"
sim_name = "antenna_array"
sim_path = Path.cwd() / "src" / "sim" / sim_name

nf2ff = analyze.read_nf2ff(sim_path / "farfield_4_4.h5")
fig = plt.figure(figsize=(12, 5))
analyze.plot_ff_2d(nf2ff, ax=fig.add_subplot(121))
analyze.plot_ff_3d(nf2ff, logscale=-20, ax=fig.add_subplot(122, projection="3d"))

In [ ]:
nf2ff = analyze.read_nf2ff(sim_path / "farfield_1_1.h5")
fig = plt.figure(figsize=(12, 5))
analyze.plot_ff_2d(nf2ff, ax=fig.add_subplot(121))
analyze.plot_ff_3d(nf2ff, logscale=-20, ax=fig.add_subplot(122, projection="3d"))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

phi, theta = nf2ff["phi"], nf2ff["theta"]
xn, yn = 4, 4
ant_midX = np.array([144.0, 48.0, -48.0, -144.0])
ant_midY = np.array([180.0, 60.0, -60.0, -180.0])
C0 = 299792458
f_res = 2.44e-9

In [ ]:
def calculate_array_factor(
    theta,
    phi,
    xn,
    yn,
    dx,
    dy,
    phase_shift_x=0,
    phase_shift_y=0,
):
    """Calculate array factor for a rectangular array

    Args:
        theta, phi: Far-field angles in degrees
        xn, yn: Number of elements in x and y directions
        dx, dy: Element spacing in meters
        phase_shift_x, phase_shift_y: Progressive phase shift between elements
    """
    theta_rad = theta
    phi_rad = phi

    # Convert to directional cosines
    u = np.sin(theta_rad) * np.cos(phi_rad)
    v = np.sin(theta_rad) * np.sin(phi_rad)

    # Calculate array factor
    AF = np.zeros((len(theta), len(phi)), dtype=complex)
    k0 = 2 * np.pi * f0 / C0

    for i in range(xn):
        for j in range(yn):
            x_pos = (i - (xn - 1) / 2) * dx
            y_pos = (j - (yn - 1) / 2) * dy
            phase = k0 * (x_pos * u + y_pos * v) + i * phase_shift_x + j * phase_shift_y
            AF += np.exp(1j * phase)

    return AF


# Load simulation results
nf2ff = analyze.read_nf2ff(sim_path / "farfield_1_1.h5")
E_theta_single, E_phi_single, theta, phi = (
    nf2ff["E_theta"],
    nf2ff["E_phi"],
    nf2ff["theta"],
    nf2ff["phi"],
)
nf2ff = analyze.read_nf2ff(sim_path / "farfield_4_4.h5")
E_theta_array, E_phi_array = nf2ff["E_theta"], nf2ff["E_phi"]

# Calculate array factor
patch_width = 32  # patch width (resonant length) in x-direction
patch_length = 40  # patch length in y-direction

# Convert mm to meters
dx = patch_width * 3e-3
dy = patch_length * 3e-3

# f0 = 2.45e9
f0 = 3e9
AF = calculate_array_factor(theta, phi, 4, 4, dx, dy)

# Calculate pattern from single element and array factor
E_total_theory = (E_theta_single + E_phi_single) * AF
E_total_sim = E_theta_array + E_phi_array

# Normalize patterns
E_total_theory = E_total_theory / np.max(np.abs(E_total_theory))
E_total_sim = E_total_sim / np.max(np.abs(E_total_sim))

# Plot comparison

# E-plane cut (phi = 0)
phi_idx = np.where(phi == 0)[0][0]
plt.figure(figsize=(10, 6))
plt.plot(
    np.degrees(theta),
    np.squeeze(20 * np.log10(np.abs(E_total_sim[:, phi_idx]))),
    label="Full-wave simulation",
)
plt.plot(
    np.degrees(theta),
    np.squeeze(20 * np.log10(np.abs(E_total_theory[:, phi_idx]))),
    "--",
    label="Pattern multiplication",
)
plt.xlabel("Theta (degrees)")
plt.ylabel("Normalized magnitude (dB)")
plt.title("E-plane Pattern Comparison")
plt.grid(True)
plt.legend()
plt.ylim(-40, 0)
plt.savefig("pattern_comparison.png")
plt.show()


In [ ]:
# calculate lambda
lambda0 = C0 / f0
lambda0

# compare the antenna spacing with the wavelength
d = 0.5 * lambda0
d, dx, d / dx

In [ ]:
# setup FDTD parameter & excitation function
f0 = 0  # center frequency
fc = 3e9  # 20 dB corner frequency

In [ ]:
np.linspace(max(1e9, f0 - fc), f0 + fc, 501)